# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [16]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [17]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [18]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10#experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [19]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    img_temp = imresize(image,(120,120))
                    
                    batch_data[folder,idx,:,:,0] = (img_temp[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (img_temp[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (img_temp[:,:,0])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    img_temp = imresize(image,(120,120))
                    
                    batch_data[folder,idx,:,:,0] = (img_temp[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (img_temp[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (img_temp[:,:,2])/255
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [20]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [21]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization
from keras.layers import Activation, Dropout, ZeroPadding3D
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
# from keras.layers.recurrent import LSTM, GRU, Bidirectional
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN

#write your model here

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]
channel = 3

# Input
input_shape=(x,y,z,channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [22]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_9 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_10 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_6 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
#CNN2D + LSTM
model_lstm = Sequential()



model_lstm.add(TimeDistributed(Conv2D(16, (3, 3), strides=(2, 2),
    activation='relu', padding='same'), input_shape=input_shape))
model_lstm.add(TimeDistributed(BatchNormalization()))
model_lstm.add(TimeDistributed(Conv2D(16, (3,3),
    kernel_initializer="he_normal", activation='relu')))
model_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_lstm.add(TimeDistributed(Conv2D(32, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(BatchNormalization()))
model_lstm.add(TimeDistributed(Conv2D(32, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_lstm.add(TimeDistributed(Conv2D(64, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(BatchNormalization()))
model_lstm.add(TimeDistributed(Conv2D(64, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_lstm.add(TimeDistributed(Conv2D(128, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(BatchNormalization()))
model_lstm.add(TimeDistributed(Conv2D(128, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_lstm.add(TimeDistributed(Conv2D(256, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(BatchNormalization()))
model_lstm.add(TimeDistributed(Conv2D(256, (3,3),
    padding='same', activation='relu')))
model_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_lstm.add(TimeDistributed(Flatten()))

model_lstm.add(Dropout(0.5))
model_lstm.add(LSTM(128, return_sequences=False, dropout=0.5))
model_lstm.add(Dense(nb_dense[2], activation='softmax'))

In [24]:
optimiser = Adam() #write your optimizer
model_lstm.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_lstm.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_22 (TimeDis (None, 30, 60, 60, 16)    448       
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 60, 60, 16)    64        
_________________________________________________________________
time_distributed_24 (TimeDis (None, 30, 58, 58, 16)    2320      
_________________________________________________________________
time_distributed_25 (TimeDis (None, 30, 29, 29, 16)    0         
_________________________________________________________________
time_distributed_26 (TimeDis (None, 30, 29, 29, 32)    4640      
_________________________________________________________________
time_distributed_27 (TimeDis (None, 30, 29, 29, 32)    128       
_________________________________________________________________
time_distributed_28 (TimeDis (None, 30, 29, 29, 32)    9248      
__________

In [25]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [26]:
model_name = 'model_init_Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.20, patience=5, cooldown=1, verbose=1, mode='auto', min_delta=0.0001) 
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [28]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  Project_data/train ; batch size = 10
10
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.9049 - categorical_accuracy: 0.2846Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 80s 1s/step - loss: 1.9008 - categorical_accuracy: 0.2777 - val_loss: 1.4362 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_Conv3D_2020-03-3012_00_31.330215/model-00001-1.90164-0.28054-1.43619-0.34000.h5
Epoch 2/50
67/67 [==============================] - 25s 372ms/step - loss: 1.5613 - categorical_accuracy: 0.3134 - val_loss: 1.3160 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to model_init_Conv3D_2020-03-3012_00_31.330215/model-00002-1.56133-0.31343-1.31602-0.39000.h5
Epoch 3/50
67/67 [==============================] - 26s 395ms/step - loss: 1.5249 - categorical_accuracy: 0.2985 - val_loss: 1.3158 - val_categorical_accuracy: 0.3900

Epoch 00003: saving model to model_init_Conv3D_2020-03-3012_00_31.330215/model-00003-1.52489-0.29851-1.31575-0.39000.h5
Epoch 4/50
67/67 [==============================] - 26s 382ms/step - loss: 1.4593 - categorical_accuracy: 0.3881 - val_loss: 1.5927 - val_categorical_

In [29]:
model_name = 'model_init_LSTM' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.20, patience=5, cooldown=1, verbose=1, mode='auto', min_delta=0.0001) 
callbacks_list = [checkpoint, LR]

In [30]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [31]:
model_lstm.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 31s 457ms/step - loss: 1.6723 - categorical_accuracy: 0.2438 - val_loss: 1.4167 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_LSTM_2020-03-3012_00_31.330215/model-00001-1.67226-0.24378-1.41672-0.39000.h5
Epoch 2/50
67/67 [==============================] - 24s 356ms/step - loss: 1.6907 - categorical_accuracy: 0.2289 - val_loss: 1.7071 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_LSTM_2020-03-3012_00_31.330215/model-00002-1.69066-0.22886-1.70713-0.23000.h5
Epoch 3/50
67/67 [==============================] - 28s 413ms/step - loss: 1.7386 - categorical_accuracy: 0.1841 - val_loss: 1.7042 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_LSTM_2020-03-3012_00_31.330215/model-00003-1.73858-0.18408-1.70421-0.22000.h5
Epoch 4/50
67/67 [==============================] - 28s 413ms/step - loss: 1.6757 - categorical_accuracy: 0.2736 - val_loss: 1.7126 - val_categorical_acc